In [16]:
## FYI
# M (int): Number of sources
# K (int): Number of time slots per experiment
# G (int): Number of independent experiments (for averaging)
# alpha_i (array-like, shape (M, )): Weights for objective J. Default = (M+1-i)/M
# p_i : (array-like, shape (M, )): Bernoulli arrival probabilities per source. Default = i/M

## Simulation Result: Greedy-MaxAge == WeightedGreedy(w=1) performs better compared to Random Scheduling in these simulation settings.

In [17]:
import numpy as np
def simulate_greedymaxage(M, K = 10000, G = 100, alpha_i = None, p_i = None, seed = 0):
    rng = np.random.default_rng(seed)
    i_idx = np.arange(1, M+1)
    if alpha_i is None:
        alpha_i = (M+1-i_idx) / M
    else:
        alpha_i = np.asarray(alpha_i, dtype=float)

    if p_i is None:
        p_i = i_idx/M
    else:
        p_i = np.asarray(p_i, dtype=float)

    assert alpha_i.shape == (M, )
    assert p_i.shape == (M, )

    J_avg_exp = 0.0

    for g in range(G):
        A = np.ones(M, dtype=int)

        buf_age = np.zeros(M, dtype=int)
        has_pkt = np.zeros(M, dtype=bool)

        J_sum = 0.0

        for t in range(K):
            A += 1
            arrivals = rng.random(M) < p_i
            buf_age[arrivals] = 1
            has_pkt[arrivals] = True
            buf_age[~arrivals & has_pkt] += 1

            if np.any(has_pkt):
                metric = A.astype(float).copy()
                metric[~has_pkt] = -1.0
                s = int(np.argmax(metric))
                A[s] = buf_age[s]
                has_pkt[s] = False
                buf_age[s] = 0
            
            J_sum += np.sum(alpha_i*A)
        
        J_avg = J_sum/K
        J_avg_exp += J_avg
    
    J_avg_exp /= G
    return J_avg_exp

In [18]:
def simulate_random(M, K=10000, G=100, alpha_i=None, p_i=None, seed=0):
    rng = np.random.default_rng(seed)

    i_idx = np.arange(1, M+1)
    if alpha_i is None:
        alpha_i = (M + 1 - i_idx)/M
    else:
        alpha_i = np.asarray(alpha_i, dtype=float)

    if p_i is None:
        p_i = i_idx / M
    else:
        p_i = np.asarray(p_i, dtype=float)

    assert alpha_i.shape == (M,)
    assert p_i.shape == (M,)

    J_avg_exp = 0.0

    for g in range(G):
        A = np.ones(M, dtype=int)
        buf_age = np.zeros(M, dtype=int)
        has_pkt = np.zeros(M, dtype=bool)

        J_sum = 0.0

        for t in range(K):
            A += 1
            arrivals = rng.random(M) < p_i
            buf_age[arrivals] = 1
            has_pkt[arrivals] = True
            buf_age[~arrivals & has_pkt] += 1

            cand = np.where(has_pkt)[0]
            if cand.size > 0:
                s = int(rng.choice(cand))

                A[s] = buf_age[s]
                has_pkt[s] = False
                buf_age[s] = 0

            J_sum += np.sum(alpha_i * A)

        J_avg = J_sum / K
        J_avg_exp += J_avg

    J_avg_exp /= G
    return J_avg_exp

In [19]:
def simulate_weightedgreedy(M, K=10000, G=100, alpha_i=None, p_i=None, weights=None, seed=0):
    rng = np.random.default_rng(seed)

    i_idx = np.arange(1, M+1)
    if alpha_i is None:
        alpha_i = (M + 1 - i_idx)/M
    else:
        alpha_i = np.asarray(alpha_i, dtype=float)

    if p_i is None:
        p_i = i_idx / M
    else:
        p_i = np.asarray(p_i, dtype=float)

    if weights is None:
        weights = np.ones(M)
    else:
        weights = np.asarray(weights, dtype=float)

    assert alpha_i.shape == (M,)
    assert p_i.shape == (M,)
    assert weights.shape == (M,)

    J_avg_exp = 0.0

    for g in range(G):
        A = np.ones(M, dtype=int)
        buf_age = np.zeros(M, dtype=int)
        has_pkt = np.zeros(M, dtype=bool)

        J_sum = 0.0

        for t in range(K):
            A += 1
            arrivals = rng.random(M) < p_i
            buf_age[arrivals] = 1
            has_pkt[arrivals] = True
            buf_age[~arrivals & has_pkt] += 1

            if np.any(has_pkt):
                metric = weights * A.astype(float)
                metric[~has_pkt] = -1.0
                s = int(np.argmax(metric))

                A[s] = buf_age[s]
                has_pkt[s] = False
                buf_age[s] = 0

            J_sum += np.sum(alpha_i * A)

        J_avg = J_sum / K
        J_avg_exp += J_avg

    J_avg_exp /= G
    return J_avg_exp

In [20]:
M = 15
K = 10000
G = 100

i = np.arange(1, M+1)
alpha_i = (M + 1 - i)/M
p_i = i / M

# All weights = 1 → AoI-aware but unweighted
weights = np.ones(M)

J_rand = simulate_random(M, K=K, G=G, alpha_i=alpha_i, p_i=p_i, seed=1)
J_gma  = simulate_greedymaxage(M, K=K, G=G, alpha_i=alpha_i, p_i=p_i, seed=1)
J_wg   = simulate_weightedgreedy(M, K=K, G=G, alpha_i=alpha_i, p_i=p_i, weights=weights, seed=1)

print("Random scheduling      J_avg_exp =", J_rand)
print("Greedy-MaxAge          J_avg_exp =", J_gma)
print("WeightedGreedy (w=1)   J_avg_exp =", J_wg)


Random scheduling      J_avg_exp = 136.38154233333336
Greedy-MaxAge          J_avg_exp = 81.61654673333332
WeightedGreedy (w=1)   J_avg_exp = 81.61654673333332
